# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = pd.DataFrame(data, columns = ['CustomerID', 'FirstName', 'LastName',
                                   'ProductID','ProductName','Quantity'])

df.groupby(['CustomerID', 'ProductName']).sum()


ProductID  Quantity
CustomerID ProductName                                           
33         Apricots - Dried                         324         1
           Assorted Desserts                        219         1
           Bandage - Flexible Neon                  273         1
           Bar Mix - Pina Colada, 355 Ml            281         1
           Beans - Kidney, Canned                   300         1
           Beef - Chuck, Boneless                   439         1
           Beef - Prime Rib Aaa                     286         1
           Beer - Original Organic Lager            432         1
           Beer - Rickards Red                      284         1
           Black Currants                           110         1
           Blackberries                             441         1
           Bread - Italian Roll With Herbs          287         1
           Bread Foccacia Whole                     378         1
           Cake - Box Window 10x10x2.5              383         1
           Cassis                                    53         1
           Cheese - Cottage Cheese                  333         1
           Crab - Imitation Flakes                   23         1
           Fondant - Icing                          387         1
           French Pastry - Mini Chocolate           214         1
           General Purpose Trigger                   91         1
           Grapes - Red                             267         1
           Grouper - Fresh                          490         2
           Hersey Shakes                            184         1
           Ice Cream Bar - Drumstick                154         1
           Lamb - Pieces, Diced                     417         1
           Lettuce - Spring Mix                     886         2
           Mangoes                                  342         1
           Muffin Batt - Choc Chk                   309         1
           Onions - Vidalia                         322         1
           Pasta - Cheese / Spinach Bauletti        348         1
...                                                 ...       ...
98200      Onion Powder                             280        25
           Orange - Canned, Mandarin                288        25
           Pasta - Detalini, White, Fresh           257        25
           Puree - Passion Fruit                    392        25
           Raspberries - Fresh                       69        25
           Rice - Long Grain                        359        25
           Rosemary - Dry                           173        25
           Rosemary - Primerba, Paste               204        50
           Rum - Coconut, Malibu                    138        25
           Sage - Ground                             38        25
           Sausage - Liver                          344        25
           Sea Bass - Whole                         554        50
           Seedlings - Mix, Organic                 390        25
           Sole - Dover, Whole, Fresh               222        25
           Soup - Campbells, Lentil                 202        50
           Soup Campbells - Italian Wedding         119        25
           Soup Knorr Chili With Beans              256        25
           Soupfoamcont12oz 112con                  263        25
           Spinach - Baby                            29        25
           Table Cloth 54x72 White                   36        25
           Tea - English Breakfast                  258        25
           Tilapia - Fillets                        357        25
           Tomatoes Tear Drop                       163        25
           Truffle Cups - Brown                     418        25
           Vaccum Bag 10x13                         381        25
           Vol Au Vents                             272        50
           Wasabi Powder                             80        25
           Wine - Fume Blanc Fetzer                  76        25
           Wine - Ha

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(df, values='Quantity', index=['ProductName'], columns=['CustomerID'],
                       aggfunc=np.sum).fillna(0)
table.head(15)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,50.0,25.0,0.0,25.0,25.0
Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
Arizona - Green Tea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
Artichokes - Jerusalem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
simil_customers= squareform(pdist(table.T, metric='euclidean'))
simil_customers.shape
step3 = pd.DataFrame(simil_customers, index=table.columns, columns=table.columns)
step3

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
464,11.090537,12.288206,11.445523,11.789826,11.704700,0.000000,12.369317,10.816654,11.489125,11.789826,...,206.637364,212.365251,225.441788,197.481645,231.136323,219.599636,217.425849,228.197283,239.006276,229.017467
477,12.409674,12.165525,12.000000,12.328828,12.328828,12.369317,0.000000,11.661904,12.206556,12.409674,...,206.329833,212.301672,224.937769,197.539869,230.427863,219.538152,216.903204,228.028507,238.949786,229.503813
639,11.045361,12.083046,11.401754,11.135529,11.135529,10.816654,11.661904,0.000000,10.630146,11.045361,...,206.620425,212.584101,225.647956,197.590486,231.121180,219.924987,217.294731,228.181945,238.468027,229.656265
649,11.269428,11.874342,11.180340,11.958261,11.789826,11.489125,12.206556,10.630146,0.000000,11.532563,...,206.879192,212.835617,225.552655,197.608198,230.811611,219.827205,217.080630,227.868383,238.692271,229.997826


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [28]:
elelegido= 649
sim649= step3[elelegido].sort_values(ascending=False)[1:6].index
sim649

Int64Index([97324, 95017, 92492, 91777, 92525], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [29]:
data_customers = data[data['CustomerID'].isin(sim649)]
data_customers

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
57,97324,Darrick,Williamson,5653021,229,Bread - Hot Dog Buns,25
58,97324,Darrick,Williamson,6196374,229,Bread - Hot Dog Buns,25
103,95017,Guillermo,Bridges,4966438,229,Bread - Hot Dog Buns,25
800,92525,Claude,Dunn,1869933,102,"Rosemary - Primerba, Paste",24
806,91777,Ramona,Cabrera,2203,102,"Rosemary - Primerba, Paste",24
1048,95017,Guillermo,Bridges,3690474,417,"Lamb - Pieces, Diced",25
1125,91777,Ramona,Cabrera,6252822,17,"Wine - White, Mosel Gold",24
1286,91777,Ramona,Cabrera,2889627,159,Eggplant - Asian,24
1419,92492,Ben,Fitzgerald,3300278,196,Longos - Grilled Salmon With Bbq,24
1606,91777,Ramona,Cabrera,3103361,69,Raspberries - Fresh,24


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [30]:
totalBuyedProducts= data_customers.groupby('ProductName', as_index=True).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)

totalBuyedProducts


,Quantity
ProductName,
"Turnip - White, Organic",146
"Beef - Tenderlion, Center Cut",122
Whmis - Spray Bottle Trigger,98
Pastry - Butterscotch Baked,98
Clam Nectar,98
Bandage - Flexible Neon,98
Squid U5 - Thailand,97
Coffee - Irish Cream,75
Bread - Hot Dog Buns,75


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [31]:
notpurchased = totalBuyedProducts.merge(table, left_on='ProductName', right_on='ProductName')
notpurchased = notpurchased[["Quantity",elelegido]]
notpurchased = notpurchased.loc[notpurchased[elelegido].isin([0])].head()
notpurchased.index

Index(['Turnip - White, Organic', 'Beef - Tenderlion, Center Cut',
       'Whmis - Spray Bottle Trigger', 'Pastry - Butterscotch Baked',
       'Clam Nectar'],
      dtype='object', name='ProductName')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.